In [10]:
import pandas as pd
import numpy as np
import torch
import os
#from utils.create_predwin import create_predwin

### Make Timeseries

In [146]:
PKL_NAMES = [i for i in os.listdir('data') if 'expt0' in i ]
PKL_NAME = PKL_NAMES[6][:-4]
combined = pd.read_pickle('data/' + PKL_NAME + '.pkl')

In [52]:
CSV_NAMES

['expt10.csv',
 'expt11.csv',
 'expt12.csv',
 'expt13.csv',
 'expt2_diff1_4hours.csv',
 'expt3_diff1_9hours.csv',
 'expt4_expt3plusdiffdiff1_9hours.csv',
 'expt5_mean_diff1_9hours.csv',
 'expt6_T0to-9mean.csv',
 'expt7_nooffarms.csv',
 'expt8_is_high_wind_season.csv',
 'expt9.csv']

In [155]:
CSV_NAMES = [i for i in os.listdir('data') if 'csv' in i ]
CSV_NAMES = [i for i in CSV_NAMES if 'expt' in i ]

In [156]:
for csv in CSV_NAMES: 
    CSV_NAME = csv[:-4]
    combined = pd.read_csv('data/' + CSV_NAME + '.csv')
    
    # Normalise
    # M = combined.values
    # M_mean = np.mean(M, axis=0)
    # M_std = np.std(M, axis=0)
    # M = (M - M_mean)/ M_std
    
    def create_predwin(a, winsize, pred_target, pred_lag, offset=0): 
        """
        a(np.array): time x dimensions 
        """
        padding = winsize + pred_lag - 1 
        until = a.shape[0] - padding + offset

        y = a[padding - offset:, pred_target]
        X = np.zeros((until, winsize, a.shape[1]))

        for i in np.arange(until): 
            X[i,:,:] = a[i:i+winsize,:]

        # when target offsets predictor
        if offset != 0: 
            X = np.delete(X,pred_target,axis=2)

        return X,y
    
    ### CREATE PREDICTION WINDOW
    WINDOW_SIZE = 64 # 128 
    LAG = 18
    X, y = create_predwin(combined.values, WINDOW_SIZE, 0, 1, offset=1) # lag is already included in the data 

    ### SPLIT TRAIN, VAL, TEST
    # Make tensors
    X,y = torch.Tensor(X), torch.Tensor(y)
    
    index = np.arange(X.shape[0])
    index_train = index[: int(0.8 * X.shape[0])]
    index_val = index[int(0.8 * X.shape[0]) : int(0.9 * X.shape[0])]
    index_test = index[int(0.9 * X.shape[0]) : ]
    
    # Old Data Prediction of New Data
    index = np.arange(X.shape[0])
    index_train = index[: int(0.8 * X.shape[0])]
    index_val = index[int(0.8 * X.shape[0]) : int(0.9 * X.shape[0])]
    index_test = index[int(0.9 * X.shape[0]) : ]
    #for i_ in [index_train, index_val]: 
    #    np.random.shuffle(i_)

    # Splitting Dataset
    train = (X[index_train, :, :], y[index_train])
    val = (X[index_val, :, :], y[index_val])
    test = (X[index_test, :, :], y[index_test])
    torch.save((train, val, test), 'data/tensors/' + CSV_NAME + '.pkl')

    persistence = y[index_test - LAG]
    torch.save(persistence, 'data/tensors/' + CSV_NAME + '_persistence.pkl')

### Loading

In [88]:
train, val, test = torch.load('data/tensors/' + PKL_NAME + '.pkl')
X_train, y_train = train 
X_val, y_val = val 
X_test, y_test = test 

In [89]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

torch.Size([24702, 64, 56])
torch.Size([3088, 64, 56])
torch.Size([3088, 64, 56])
